In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed

In [56]:
# Reading the dataset
df = pd.read_csv('Agt_weather.csv')

In [57]:
# displaying the data
df.head()

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,"Agartala, TR, India",2025-01-20T00:00:00,58.7,58.7,54.4,85.55,0.0,0,NaN,0,...,1013.0,0.0,15.0,0.0,0.0,0,10,Clear,clear-night,remote
1,"Agartala, TR, India",2025-01-20T01:00:00,58.2,58.2,54.2,86.65,0.0,0,NaN,0,...,1012.0,0.0,15.0,0.0,0.0,0,10,Clear,clear-night,remote
2,"Agartala, TR, India",2025-01-20T02:00:00,55.6,55.6,55.4,99.60,0.0,0,NaN,0,...,1011.7,0.0,0.7,0.0,0.0,0,10,Clear,fog,"42724099999,41915099999,41933099999"
3,"Agartala, TR, India",2025-01-20T03:00:00,57.1,57.1,52.8,85.46,0.0,0,NaN,0,...,1012.0,0.0,15.0,0.0,0.0,0,10,Clear,clear-night,remote
4,"Agartala, TR, India",2025-01-20T04:00:00,55.3,55.3,53.1,92.42,0.0,0,NaN,0,...,1012.0,0.0,15.0,0.0,0.0,0,10,Clear,clear-night,remote


In [58]:
# Setting the datetime column as index column
df = df.set_index("datetime")
df.index = pd.to_datetime(df.index)

In [60]:
df.head()

,name,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
datetime,,,,,,,,,,,,,,,,,,,,,
2025-01-20 00:00:00,"Agartala, TR, India",58.7,58.7,54.4,85.55,0.0,0,NaN,0,0,...,1013.0,0.0,15.0,0.0,0.0,0,10,Clear,clear-night,remote
2025-01-20 01:00:00,"Agartala, TR, India",58.2,58.2,54.2,86.65,0.0,0,NaN,0,0,...,1012.0,0.0,15.0,0.0,0.0,0,10,Clear,clear-night,remote
2025-01-20 02:00:00,"Agartala, TR, India",55.6,55.6,55.4,99.60,0.0,0,NaN,0,0,...,1011.7,0.0,0.7,0.0,0.0,0,10,Clear,fog,"42724099999,41915099999,41933099999"
2025-01-20 03:00:00,"Agartala, TR, India",57.1,57.1,52.8,85.46,0.0,0,NaN,0,0,...,1012.0,0.0,15.0,0.0,0.0,0,10,Clear,clear-night,remote
2025-01-20 04:00:00,"Agartala, TR, India",55.3,55.3,53.1,92.42,0.0,0,NaN,0,0,...,1012.0,0.0,15.0,0.0,0.0,0,10,Clear,clear-night,remote


In [61]:
# Dropping columns which are not required
df.drop(columns=['name','dew','feelslike','precip','precipprob','snow','snowdepth','windgust','winddir','solarradiation','solarenergy','uvindex','severerisk','icon','stations'], inplace=True)

In [79]:
df.head()

,temp,humidity,preciptype,windspeed,sealevelpressure,cloudcover,visibility,conditions
datetime,,,,,,,,
2025-01-20 00:00:00,58.7,85.55,0,4.3,1013.0,0.0,15.0,Clear
2025-01-20 01:00:00,58.2,86.65,0,4.7,1012.0,0.0,15.0,Clear
2025-01-20 02:00:00,55.6,99.60,0,0.0,1011.7,0.0,0.7,Clear
2025-01-20 03:00:00,57.1,85.46,0,5.1,1012.0,0.0,15.0,Clear
2025-01-20 04:00:00,55.3,92.42,0,4.3,1012.0,0.0,15.0,Clear


In [63]:
df.shape

(984, 8)

In [84]:
df.iloc[261,7]

'Partially cloudy'

In [77]:
# checking the range of conditions and converting the categorical data to numerical
arr1 = []
ct = 0
for i in range(984):
    el1 = df.iloc[i,2]
    if pd.isna(df.iloc[i,2]):
        ct += 1
        df.iloc[i,2] = 0
    else:
        df.iloc[i,2] = 1

    el2 = df.iloc[i,7]
    if el2 not in arr1:
        arr1.append(el2)

In [75]:
print(arr1)

['Clear', 'Partially cloudy', 'Overcast', 'Rain, Overcast', 'Rain, Partially cloudy', 'Rain']


In [78]:
ct

969

In [82]:
d = dict()
for i in range(len(arr1)):
    d[arr1[i]] = i

In [83]:
d

{'Clear': 0,
 'Partially cloudy': 1,
 'Overcast': 2,
 'Rain, Overcast': 3,
 'Rain, Partially cloudy': 4,
 'Rain': 5}

In [85]:
# converting condition data to numerical
for i in range(984):
    df.iloc[i,7] = d[df.iloc[i,7]]

In [86]:
df.head()

,temp,humidity,preciptype,windspeed,sealevelpressure,cloudcover,visibility,conditions
datetime,,,,,,,,
2025-01-20 00:00:00,58.7,85.55,0,4.3,1013.0,0.0,15.0,0
2025-01-20 01:00:00,58.2,86.65,0,4.7,1012.0,0.0,15.0,0
2025-01-20 02:00:00,55.6,99.60,0,0.0,1011.7,0.0,0.7,0
2025-01-20 03:00:00,57.1,85.46,0,5.1,1012.0,0.0,15.0,0
2025-01-20 04:00:00,55.3,92.42,0,4.3,1012.0,0.0,15.0,0


In [87]:
df.sample(10)

,temp,humidity,preciptype,windspeed,sealevelpressure,cloudcover,visibility,conditions
datetime,,,,,,,,
2025-02-23 06:00:00,69.8,88.35,1,0.0,1016.0,89.0,1.2,1
2025-02-06 16:00:00,82.4,48.09,0,4.7,1009.0,0.0,1.9,0
2025-01-30 13:00:00,80.6,44.83,0,9.2,1010.0,27.6,2.5,1
2025-02-17 17:00:00,81.1,53.55,0,1.6,1011.9,10.0,2.5,0
2025-02-01 15:00:00,80.6,65.56,0,5.8,1010.0,0.0,1.9,0
2025-02-13 14:00:00,86.0,48.87,0,4.5,1009.1,33.5,2.8,1
2025-02-24 05:00:00,62.9,97.60,0,0.0,1016.9,32.8,1.3,1
2025-02-07 07:00:00,62.6,77.31,0,0.0,1015.0,0.0,1.2,0
2025-02-02 21:00:00,72.2,83.15,0,0.7,1012.0,0.3,15.0,0


In [89]:
# Scaling data using MinMaxScaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

In [90]:
df_scaled

array([[0.17339667, 0.81282383, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.16152019, 0.82707254, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.09976247, 0.99481865, 0.        , ..., 0.        , 0.04666667,
        0.        ],
       ...,
       [0.52494062, 0.70531088, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.52019002, 0.71761658, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.52494062, 0.71787565, 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [91]:
# Checking missing values
np.isnan(np.sum(df_scaled))

False

In [109]:
# defining function for splitting dataset
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y, X_test, y_test = list(), list(), list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequence):
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        
        if i <= 850:
            X.append(seq_x)
            y.append(seq_y)
        else:
            X_test.append(seq_x)
            y_test.append(seq_y)
    
    return np.array(X), np.array(y), np.array(X_test), np.array(y_test)

In [114]:
X, y, X_test, y_test = split_sequence(df_scaled, 100, 12)

In [115]:
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(851, 100, 8)
(851, 12, 8)
(22, 100, 8)
(22, 12, 8)


In [117]:
n_steps_in, n_steps_out = 100, 12

In [120]:
n_features, n_epochs, n_neurons = df.columns.size, 20, 200

In [121]:
# Creating a LSTM model
model = Sequential()
model.add(LSTM(n_neurons, activation='tanh', input_shape = (n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(n_neurons, activation='tanh', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer="adam", loss = "mse")
model.fit(X[:1],y[:1], epochs=n_epochs, verbose=1)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - loss: 0.2146
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 0.1541
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.1077
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.0909
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.0926
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0798
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 0.0760
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.0745
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0711
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 0.0661
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.0611
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.0572
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 0.0537
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 0.0491
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 0.0436
Epoch 16/20
1/1 ━━━━━━

In [123]:
import pickle

In [124]:
# Dumping the model to pkl format
with open("scaler3.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [125]:
# saving the model
model.save("model_4.h5")

In [126]:
df.columns

Index(['temp', 'humidity', 'preciptype', 'windspeed', 'sealevelpressure',
       'cloudcover', 'visibility', 'conditions'],
      dtype='object')

In [127]:
df.head()

,temp,humidity,preciptype,windspeed,sealevelpressure,cloudcover,visibility,conditions
datetime,,,,,,,,
2025-01-20 00:00:00,58.7,85.55,0,4.3,1013.0,0.0,15.0,0
2025-01-20 01:00:00,58.2,86.65,0,4.7,1012.0,0.0,15.0,0
2025-01-20 02:00:00,55.6,99.60,0,0.0,1011.7,0.0,0.7,0
2025-01-20 03:00:00,57.1,85.46,0,5.1,1012.0,0.0,15.0,0
2025-01-20 04:00:00,55.3,92.42,0,4.3,1012.0,0.0,15.0,0


In [128]:
df.tail()

,temp,humidity,preciptype,windspeed,sealevelpressure,cloudcover,visibility,conditions
datetime,,,,,,,,
2025-03-01 19:00:00,80.5,69.73,0,9.2,1009.0,25.0,2.5,1
2025-03-01 20:00:00,75.5,59.43,0,7.2,1010.0,0.0,15.0,0
2025-03-01 21:00:00,73.5,77.25,0,8.9,1010.0,0.0,15.0,0
2025-03-01 22:00:00,73.3,78.20,0,9.2,1010.0,0.0,15.0,0
2025-03-01 23:00:00,73.5,78.22,0,10.1,1010.0,0.0,15.0,0


In [142]:
last_data = scaler.transform(df.tail(100))

In [152]:
predicted = model.predict(last_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [144]:
len(last_data)

100

In [145]:
last_data.shape

(100, 8)

In [149]:
last_data = last_data.reshape(1, 100, 8)

In [150]:
last_data

array([[[0.49168646, 0.20025907, 0.        , 0.32786885, 0.83333333,
         0.68      , 1.        , 0.2       ],
        [0.42517815, 0.23316062, 0.        , 0.24590164, 0.91666667,
         0.079     , 1.        , 0.        ],
        [0.40855107, 0.22098446, 0.        , 0.20765027, 0.91666667,
         0.148     , 1.        , 0.        ],
        [0.39192399, 0.22668394, 0.        , 0.27868852, 0.91666667,
         0.886     , 1.        , 0.2       ],
        [0.34916865, 0.27176166, 0.        , 0.36612022, 0.91666667,
         0.048     , 1.        , 0.        ],
        [0.3064133 , 0.29598446, 0.        , 0.3442623 , 0.83333333,
         0.        , 1.        , 0.        ],
        [0.27790974, 0.30168394, 0.        , 0.31693989, 0.83333333,
         0.        , 1.        , 0.        ],
        [0.25890736, 0.28795337, 0.        , 0.2568306 , 0.75      ,
         0.        , 1.        , 0.        ],
        [0.24228029, 0.27836788, 0.        , 0.24590164, 0.75      ,
         0.

In [154]:
scaler.inverse_transform(predicted[0])

array([[ 4.79546738e+01,  7.56726151e+01, -3.73826511e-02,
         2.17469168e+00,  1.01214606e+03,  6.47174683e+01,
         3.07859039e+00,  1.16713691e+00],
       [ 4.80327072e+01,  1.00480782e+02, -3.67357284e-02,
         3.03275990e+00,  1.01413159e+03,  9.20330734e+01,
         3.61117959e+00,  1.74169087e+00],
       [ 5.05613251e+01,  1.07828262e+02, -2.64448002e-02,
         3.35238218e+00,  1.01475500e+03,  9.47618027e+01,
         3.10992789e+00,  1.90915477e+00],
       [ 5.41251869e+01,  1.05422882e+02, -2.24773511e-02,
         3.49087882e+00,  1.01461145e+03,  8.36243668e+01,
         2.30410242e+00,  1.80623484e+00],
       [ 5.77737808e+01,  9.84047089e+01, -2.86877938e-02,
         3.60467172e+00,  1.01412061e+03,  6.65168228e+01,
         1.55871630e+00,  1.54989147e+00],
       [ 6.10570488e+01,  8.99367142e+01, -4.20092493e-02,
         3.74470782e+00,  1.01353369e+03,  4.84719963e+01,
         1.01919031e+00,  1.23062980e+00],
       [ 6.38373833e+01,  8.178050

In [155]:
with open("last_data.pkl", "wb") as f:
    pickle.dump(last_data, f)

In [156]:
last_data

array([[[0.49168646, 0.20025907, 0.        , 0.32786885, 0.83333333,
         0.68      , 1.        , 0.2       ],
        [0.42517815, 0.23316062, 0.        , 0.24590164, 0.91666667,
         0.079     , 1.        , 0.        ],
        [0.40855107, 0.22098446, 0.        , 0.20765027, 0.91666667,
         0.148     , 1.        , 0.        ],
        [0.39192399, 0.22668394, 0.        , 0.27868852, 0.91666667,
         0.886     , 1.        , 0.2       ],
        [0.34916865, 0.27176166, 0.        , 0.36612022, 0.91666667,
         0.048     , 1.        , 0.        ],
        [0.3064133 , 0.29598446, 0.        , 0.3442623 , 0.83333333,
         0.        , 1.        , 0.        ],
        [0.27790974, 0.30168394, 0.        , 0.31693989, 0.83333333,
         0.        , 1.        , 0.        ],
        [0.25890736, 0.28795337, 0.        , 0.2568306 , 0.75      ,
         0.        , 1.        , 0.        ],
        [0.24228029, 0.27836788, 0.        , 0.24590164, 0.75      ,
         0.